In [90]:
import sys 
import csv
sys.path.append("..")
from src.utils.constantes import DATA_CLEAN_PATH
from src.utils.funciones import porcentaje



In [91]:
def separar_por_trimestre(dict_anios): 
    dict_final = {}
    for anio in dict_anios:
        dict_temporal = {}
        
        for filas in dict_anios[anio]:
            trimestre = filas[2]
            if trimestre not in dict_temporal: 
                dict_temporal[trimestre] = []
            dict_temporal[trimestre].append(filas)

        dict_final[anio] = dict_temporal
    return dict_final


def agrupar_por_anio_y_trimestre(filas, col_anio=1):
    grupos = {}
    for fila in filas:
        anio = fila[col_anio]
        if anio not in grupos:
            grupos[anio] = []
        grupos[anio].append(fila)
    grupos_final = separar_por_trimestre(grupos)
    return grupos_final

def agrupar_por_aglomerado(filas, col_aglomerado):
    grupos = {}
    for fila in filas:
        aglo = fila[col_aglomerado]
        if aglo not in grupos:
            grupos[aglo] = []
        grupos[aglo].append(fila)
    return grupos

def obtener_ultimo_archivo (dict_trimestres): 
    max_anio = max(dict_trimestres.keys())
    max_trimestre = max(dict_trimestres[max_anio].keys())
    return dict_trimestres[max_anio][max_trimestre]
    


In [92]:
# funciones a agregar a Funciones.py
def nombre_aglomerado(codigo):
    """
    Devuelve el nombre del aglomerado a partir de su código.
    """
    dic_aglomerados = {
        '2': 'Gran La Plata',
        '3': 'Bahía Blanca - Cerri',
        '4': 'Gran Rosario',
        '5': 'Gran Santa Fé',
        '6': 'Gran Paraná',
        '7': 'Posadas',
        '8': 'Gran Resistencia',
        '9': 'Comodoro Rivadavia - Rada Tilly',
        '10': 'Gran Mendoza',
        '12': 'Corrientes',
        '13': 'Gran Córdoba',
        '14': 'Concordia',
        '15': 'Formosa',
        '17': 'Neuquén - Plottier',
        '18': 'Santiago del Estero - La Banda',
        '19': 'Jujuy - Palpalá',
        '20': 'Río Gallegos',
        '22': 'Gran Catamarca',
        '23': 'Gran Salta',
        '25': 'La Rioja',
        '26': 'Gran San Luis',
        '27': 'Gran San Juan',
        '29': 'Gran Tucumán - Tafí Viejo',
        '30': 'Santa Rosa - Toay',
        '31': 'Ushuaia - Río Grande',
        '32': 'Ciudad Autónoma de Buenos Aires',
        '33': 'Partidos del GBA',
        '34': 'Mar del Plata',
        '36': 'Río Cuarto',
        '38': 'San Nicolás - Villa Constitución',
        '91': 'Rawson - Trelew',
        '93': 'Viedma - Carmen de Patagones'
    }
    return dic_aglomerados.get(codigo)

def suma_ponderada(filas, condicion, col_pondera):
    """
    Calcula la suma ponderada de una determinada columna, considerando solo las filas que cumplen una condición.

    Parámetros:
    - filas: lista de listas, donde cada sublista representa una fila de datos.
    - condicion: función que recibe una fila y devuelve True si debe ser considerada en la suma.
    - col_pondera: índice entero que indica la posición de la columna que contiene el valor ponderador.

    Retorna:
    - Un número entero que representa la suma ponderada de las filas que cumplen con la condición.
    """
    total = 0
    for fila in filas:
        if condicion(fila):
            total += int(fila[col_pondera])
    return total

Abriendo archivo individuos

In [93]:
archivo_clean_path = DATA_CLEAN_PATH / "usu_clean_individual.csv"

archivo_individuos = archivo_clean_path.open("r",encoding="utf-8") 
reader = csv.reader(archivo_individuos,delimiter=";")
header = next(reader)

new_list_i = []

for elem in reader:
    new_elem = elem[0].split(',')
    new_list_i.append(new_elem)

lista_filas_individual = new_list_i[:]

aglomerados_individuos = agrupar_por_aglomerado(new_list_i,8)
años_trimestres_individuos = agrupar_por_anio_y_trimestre(new_list_i)

ultimos_individuos = obtener_ultimo_archivo(años_trimestres_individuos)

ultimos_individuos_aglomerados = agrupar_por_aglomerado(ultimos_individuos,8)


Abriendo archivo hogares


In [94]:
archivo_clean_path = DATA_CLEAN_PATH / "usu_clean_hogar.csv"

archivo_hogares = archivo_clean_path.open("r",encoding="utf-8") 
reader = csv.reader(archivo_hogares,delimiter=";")
header = next(reader)

new_list_h = []

for elem in reader:
    new_elem = elem[0].split(',')
    new_list_h.append(new_elem)

lista_filas_hogares = new_list_h[:]


aglomerados_hogares = agrupar_por_aglomerado(new_list_h,7)
anios_trimestres_hogares = agrupar_por_anio_y_trimestre(new_list_h)

ultimos_hogares = obtener_ultimo_archivo(anios_trimestres_hogares)

ultimos_hogares_aglomerados = agrupar_por_aglomerado(ultimos_hogares,7)


1- A partir de la información de cada año contenida en el dataset se debe informar, año tras año, el porcentaje de personas mayores a 6 años capaces e incapaces de leer y escribir.
Importante: tomar la información del último trimestre de cada año.

### Tabla individual  
1- ANO4 N (4) Año de relevamiento (4 dígitos)

2- TRIMESTRE N (1) Ventana de observación  
- = 1° trimestre  
- = 2° trimestre  
- = 3° trimestre  
- = 4° trimestre  

9- PONDERA N (6) Ponderación  

12- CH05 date Fecha de nacimiento
(día, mes y año) --> ver como esta dado el dato

13- CH06 N (2) ¿Cuántos años cumplidos tiene?

16- CH09 N (1) ¿Sabe leer y escribir?
1 = Sí
2 = No
3 = Menor de 2 años

________________________________________________________
Columnas usadas: 2 - 1 - 9 - 13 - 16


In [95]:
def contar_alfabetismo(filas):
    """
    Recorre las filas y calcula los totales ponderados de:
    alfabetos, analfabetos, casos desconocidos y población mayor a 6 años.
    """
    
    alfabetos = analfabetos = desconocido = poblacion = 0

    for elem in filas:
        pondera = int(elem[9]) # es elem[8] no 9, pq es lista y empiezan en 0
        edad = int(elem[13])
        if edad >= 6:
            poblacion += pondera
            if elem[16] == "1":
                alfabetos += pondera
            elif elem[16] == "2":
                analfabetos += pondera
            else:
                desconocido += pondera
    return alfabetos, analfabetos, desconocido, poblacion

def calcular_porcentajes(alfabetos, analfabetos, desconocido, poblacion):
    """
    Calcula los porcentajes de alfabetos, analfabetos y desconocidos sobre la población total.
    """
    return (
        porcentaje(alfabetos, poblacion),
        porcentaje(analfabetos, poblacion),
        porcentaje(desconocido, poblacion),
    )

def obtener_ultimo_trimestre(dict_anio_trimestre,anio):
    return max(dict_anio_trimestre[anio].keys())


def analizar_analfabetismo(dict_anio_trimestre):
    """
    Para cada año, calcula e imprime el último trimestre disponible y los
    porcentajes de alfabetos, analfabetos y casos desconocidos en población > 6 años.
    """
    for anio in dict_anio_trimestre:
        trimestre = obtener_ultimo_trimestre(dict_anio_trimestre, anio)
        filas = dict_anio_trimestre[anio][trimestre]
        alfabetos, analfabetos, desconocido, poblacion = contar_alfabetismo(filas)
        por_alfa, por_ana, por_desc = calcular_porcentajes(alfabetos, analfabetos, desconocido, poblacion)

        print(f"Año {anio}, Trimestre {trimestre}")
        print(f"  - Alfabetismo: {por_alfa:.2f}%")
        print(f"  - Analfabetismo: {por_ana:.2f}%")
        print(f"  - Desconocido: {por_desc:.2f}%\n")

def actividad_1(individuos):
    analizar_analfabetismo(individuos)



In [96]:
actividad_1(años_trimestres_individuos)

Año 2024, Trimestre 2
  - Alfabetismo: 98.57%
  - Analfabetismo: 1.39%
  - Desconocido: 0.04%

Año 2023, Trimestre 4
  - Alfabetismo: 98.47%
  - Analfabetismo: 1.46%
  - Desconocido: 0.07%



2. A partir de un año y trimestre elegido por el usuario informar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior.

#### Tabla individual  
1- ANO4 N (4) Año de relevamiento (4 dígitos)

2- TRIMESTRE N (1) Ventana de observación  
- = 1° trimestre  
- = 2° trimestre  
- = 3° trimestre  
- = 4° trimestre  

9- PONDERA N (6) Ponderación  

19- CH12 N (2) ¿Cuál es el nivel más alto que cursa o cursó?
- 1 = Jardín/preescolar
- 2 = Primario
- 3 = EGB
- 4 = Secundario
- 5 = Polimodal
- 6 = Terciario
- 7 = Universitario
- 8 = Posgrado universitario
- 9 = Educación especial (discapacitado)

22- CH15 N (1) ¿Dónde nació?
1. En esta localidad
2. En otra localidad de esta provincia
3. En otra provincia (especificar)
4. En un país limítrofe (especificar: Brasil, Bolivia, Chile, Paraguay, Uruguay)
5. En otro país (especificar)
9. Ns/Nr

26- NIVEL_ED N (1) Nivel educativo
- 1 = Primario incompleto (incluye educación especial)
- 2 = Primario completo
- 3 = Secundario incompleto
- 4 = Secundario completo
- 5 = Superior universitario incompleto
- 6 = Superior universitario completo
- 7 = Sin instrucción
- 9 = Ns/Nr

______________________________
Columnas usadas: 1 - 2 - 9 - 19 - 22

In [97]:
def es_extranjero(valor_ch15):
    return valor_ch15 in ['4', '5']

def es_universitario(valor_ch12):
    return valor_ch12 in ['7', '8']

def activ2 (dict_trimestres,anio_elegido= 0,trimestre_elegido= 0):
    try:
        poblacion, ext_univ = 0, 0
        for elem in dict_trimestres[anio_elegido][trimestre_elegido]:
            pondera = int(elem[8])
            poblacion += pondera
            if not es_extranjero(elem[22]):
                continue 
            if not es_universitario(elem[19]):
                continue 
            ext_univ += pondera
        return porcentaje(ext_univ,poblacion)
    
    except KeyError: 
        print(f'Valores no encontrados en el diccionario')

In [98]:
def contar_extranjeros_universitarios(filas):
    # Calcula la cantidad de extranjeros, universitarios y ambos combinados
    total_extranjeros = total_universitarios = total_universitarios_extranjeros = 0

    for elem in filas:
        pondera = int(elem[9])
        ch12 = elem[19]  # nivel educativo
        ch15 = elem[22]  # lugar de nacimiento

        if es_extranjero(ch15):
            total_extranjeros += pondera
            if es_universitario(ch12):
                total_universitarios_extranjeros += pondera

        if es_universitario(ch12):
            total_universitarios += pondera

    return total_extranjeros, total_universitarios, total_universitarios_extranjeros

Se toma el porcentaje de universitarios recibidos sobre la poblacion total, a chequear si es sobre la poblacion de extranjeros

In [99]:
anio_elegido = input("ingrese el año a buscar")
trimestre_elegido = input("ingrese el trimestre a buscar")
porcentaje_ext_univ = activ2(años_trimestres_individuos,anio_elegido,trimestre_elegido)
if porcentaje_ext_univ: 
    print(f'el porcentaje de personas extranjeras universitarias es: {porcentaje_ext_univ:.2f}')

Valores no encontrados en el diccionario


3. A partir de la información contenida en el dataset informar el año y trimestre donde hubo menor desocupación.

Criterio usado:  
Poblacion en condiciones de trabajar: aproximadamente entre 18 y 65 años varia dependiendo del sexo.  
Población Activa: incluye población ocupada y desempleados (desocupados, personas que no tienen trabajo aunque lo buscan activamente).  
Población Inactiva: gente que no tiene trabajo pero tampoco lo busca.  

Para el ejercicio se cosiderara:  
Población activa = Ocupados + Desocupados  
Tasa de paro o desempleo = (Población desocupada / Población activa) x 100

In [100]:
def calcular_tasa_desempleo (dict_trimestres):
    dict_tasa = {}
    for anio in dict_trimestres:
        if anio not in dict_tasa:
            dict_tasa[anio] = {}
        
        for trimestre in dict_trimestres[anio]:
            
            if trimestre not in dict_tasa[anio]:
                dict_tasa[anio][trimestre] = 0.0
            
            ocupados, desocupados = 0, 0
            for elem in dict_trimestres[anio][trimestre]: 
                pondera = int(elem[8])
                if elem[27] == "1":
                    ocupados += pondera
                elif elem[27] == "2":
                    desocupados += pondera
            
            activos = desocupados + ocupados
            dict_tasa[anio][trimestre] = porcentaje(desocupados,activos)
    return dict_tasa

def calcular_menor_tasa_desempleo(dict_tasa):
    min_tasa = 100
    for anio, trimestres in dict_tasa.items():
        for trimestre, tasa in trimestres.items():
            if tasa < min_tasa:
                min_tasa = tasa
                max_anio = anio
                max_trimestre = trimestre
    return max_anio, max_trimestre, min_tasa

def activ_3(dict_trimestres):
    dict_tasa = calcular_tasa_desempleo(dict_trimestres)
    max_anio, max_trimestre, max_tasa = calcular_menor_tasa_desempleo(dict_tasa)
    print(f"Menor tasa de desempleo: {max_tasa:.2f}% en año {max_anio}, trimestre {max_trimestre}")



In [101]:
activ_3(años_trimestres_individuos)

Menor tasa de desempleo: 4.71% en año 2023, trimestre 4


Abriendo el archivo Hogares



4.  Ranking de los 5 aglomerados con mayor porcentaje de hogares con dos o más 
ocupantes con estudios universitarios o superiores finalizados. Información obtenida 
a partir del par de archivos más recientes. 

In [102]:
def tiene_dos_ocupantes (elem): 
    return int(elem[64]) >= 2 
def es_universitario (elem): 
    return elem[26] in ["5","6"]

def activ4 (dict_trimestres_i, dict_trimestres_h):
# busca el ultimo archivo de Hogares y Individuos 

    ## busca el año mas reciente de cada uno de los diccionarios (divididos por triemstre)
    max_anio_i = max(dict_trimestres_i.keys())

    max_anio_h = max(dict_trimestres_h.keys())

    ## busca el trimestre mas reciente para cada uno de los diccionarios (en el año maximo obtenido anteriormmnte)
    max_tri_i = max(dict_trimestres_i[max_anio_i].keys())

    max_tri_h = max(dict_trimestres_h[max_anio_h].keys())

    ## guarda los hogares e individuos encuestados del periodo mas reciente (trimestre y año tomados anteriormente)
    ultimos_hogares = dict_trimestres_h[max_anio_h][max_tri_h]

    ultimos_individuos  = dict_trimestres_i[max_anio_i][max_tri_i]
    
    ## declarando listas y dicts
    hogares_CODUSU, individuos_CODUSU, hogares, individuos = [], [], [], [] 

    poblacion_total_aglomerado, encuestas_finales, porcentajes_finales = {}, {}, {}

# busca dentro de los hogares e individuos mas recientes si cumplen con las condiciones

    ## guarda los hogares que cumplen con la condicion y su respectivo CODUSU 
    for elem in ultimos_hogares: 
        if not tiene_dos_ocupantes(elem):
            continue
        hogares.append(elem)
        hogares_CODUSU.append(elem[0]) 

    ## guarda los individuos que cumplen con la condicion y su respectivo CODUSU 
    for elem in ultimos_individuos:
        if not es_universitario(elem): 
            continue 
        individuos.append(elem)
        individuos_CODUSU.append(elem[0])

# busca dentro de los hogares e intenta encontrar los que cumplen las condiciones de individuos y hogares

    for elem in hogares:
    
    ## añade lo que pondera cada aglomerado en un aglomerado_pondera
        if elem[7] not in poblacion_total_aglomerado:
            poblacion_total_aglomerado[elem[7]] = 0

        poblacion_total_aglomerado[elem[7]] += int(elem[8])

    ## si no cumple la condicion de individuos se ignora ya cumple con la condicion de hogares, por ende si cumple esta cumple con ambas
        if elem[0] not in individuos_CODUSU: 
            continue 

    ## añade las encuestas finales( que cumplen con ambas condiciones) a encuestas_finales, separado por aglomerado 
        if elem[7] not in encuestas_finales:
            encuestas_finales[elem[7]] = []

        encuestas_finales[elem[7]].append(elem)

    for aglomerado in encuestas_finales:
    ## crea un diccionario dividido por aglomerado con lo que ponderan las encuestas finales 
        poblacion_final_aglomerado = 0

        if aglomerado not in porcentajes_finales:
            porcentajes_finales[aglomerado] = 0 

        for elem in encuestas_finales[aglomerado]: 
            poblacion_final_aglomerado += int(elem[8])
        
    ## porcentaje de hogares que cumplen con las condiciones, dividido por aglomerado 
        porcentajes_finales[aglomerado] = porcentaje(poblacion_final_aglomerado, poblacion_total_aglomerado[aglomerado])
    

    ## encuentra el aglomerado con el mayor porcentaje 
    max_porcentaje = 0 
    max_aglomerado = 0 
    for aglomerado in porcentajes_finales: 
        if porcentajes_finales[aglomerado] > max_porcentaje:
            max_aglomerado = aglomerado
            max_porcentaje = porcentajes_finales[aglomerado]
        
    print(f'el aglomerado con mayor porcentaje fue el: {max_aglomerado} con un porcentaje de {max_porcentaje:.2f}')



In [103]:
activ4(años_trimestres_individuos,anios_trimestres_hogares)

el aglomerado con mayor porcentaje fue el: 32 con un porcentaje de 72.42


- 5. Informar para cada aglomerado el porcentaje de viviendas ocupadas por sus
propietarios.

In [104]:
def porcentaje_propietarios_por_aglomerado(dict_aglomerados, col_aglomerado=7, col_pondera=8, col_tenencia=37):
    
    resultados = {}
    
    for aglomerado in dict_aglomerados:
        for elem in dict_aglomerados[aglomerado]:
            try:
                pondera = int(elem[col_pondera])
                tenencia = elem[col_tenencia]

                if aglomerado not in resultados:
                    resultados[aglomerado] = {'total': 0, 'propietarios': 0}
        
                if tenencia in ('1', '2'):  # Propietario total o parcial
                    resultados[aglomerado]['propietarios'] += pondera
            
                resultados[aglomerado]['total'] += pondera 
            except:
                continue
    # Calcular porcentaje para cada aglomerado
    for aglomerado, valores in sorted(resultados.items()):
        
        porcentaje_prop = porcentaje(valores["propietarios"], valores["total"])
        
        print(f"Aglomerado {aglomerado}: {porcentaje_prop:.2f}% viviendas de propietarios")



In [105]:
porcentaje_propietarios_por_aglomerado(aglomerados_hogares)

Aglomerado 10: 60.77% viviendas de propietarios
Aglomerado 12: 69.69% viviendas de propietarios
Aglomerado 13: 51.33% viviendas de propietarios
Aglomerado 14: 74.32% viviendas de propietarios
Aglomerado 15: 77.97% viviendas de propietarios
Aglomerado 17: 65.89% viviendas de propietarios
Aglomerado 18: 85.90% viviendas de propietarios
Aglomerado 19: 80.18% viviendas de propietarios
Aglomerado 2: 66.95% viviendas de propietarios
Aglomerado 20: 50.45% viviendas de propietarios
Aglomerado 22: 83.34% viviendas de propietarios
Aglomerado 23: 66.98% viviendas de propietarios
Aglomerado 25: 72.74% viviendas de propietarios
Aglomerado 26: 63.48% viviendas de propietarios
Aglomerado 27: 64.93% viviendas de propietarios
Aglomerado 29: 67.26% viviendas de propietarios
Aglomerado 3: 62.95% viviendas de propietarios
Aglomerado 30: 59.61% viviendas de propietarios
Aglomerado 31: 52.61% viviendas de propietarios
Aglomerado 32: 53.63% viviendas de propietarios
Aglomerado 33: 71.48% viviendas de propiet

6. Informar el aglomerado con mayor cantidad de viviendas con más de dos ocupantes
y sin baño. Informar también la cantidad de ellas.

In [106]:
def aglomerado_mas_viviendas_sin_banio(fila, col_aglomerado=7, col_pondera=8, col_banio=19, col_miembros=64):
    """
    Recorre todas las filas y devuelve el aglomerado con más viviendas sin baño
    y más de dos ocupantes para un año y trimestre específicos.
    """
    conteo = {}

    for encuesta in fila:

        aglomerado = encuesta[col_aglomerado]
        pondera = int(encuesta[col_pondera])
        tiene_banio = encuesta[col_banio]
        miembros = int(encuesta[col_miembros])

        if tiene_banio == '2' and miembros > 2:

            if aglomerado not in conteo:
                conteo[aglomerado] = 0
            conteo[aglomerado] += pondera

    if not conteo:
        return None

    aglomerado_max = max(conteo, key=conteo.get)
    cantidad_max = conteo[aglomerado_max]
    return aglomerado_max, cantidad_max

def actividad_6(ultimos_hogares):
    """
    Imprime el aglomerado con más viviendas sin baño y más de dos ocupantes
    del último período del dataset, incluyendo el código, nombre, cantidad,
    año y trimestre.
    """
    anio = ultimos_hogares[1][1]
    trimestre = ultimos_hogares[1][2]

    # Obtener el aglomerado con más viviendas sin baño y más de 2 ocupantes
    resultado = aglomerado_mas_viviendas_sin_banio(ultimos_hogares)

    if resultado is None:
        print(f"No se encontraron viviendas sin baño con más de 2 ocupantes´")

    codigo, cantidad = resultado
    nombre = nombre_aglomerado(codigo)

    print(f"Para el año {anio} trimestre {trimestre} :")
    print(f"Aglomerado {codigo} - {nombre}")
    print(f"Cantidad de viviendas sin baño con más de 2 ocupantes: {cantidad}")


In [107]:
actividad_6(ultimos_hogares)

Para el año 2024 trimestre 2 :
Aglomerado 33 - Partidos del GBA
Cantidad de viviendas sin baño con más de 2 ocupantes: 10624


### Consulta 7
7. Informar para cada aglomerado el porcentaje de personas que hayan cursado al
menos en nivel universitario o superior.


In [108]:
def porcentaje_universitarios_por_aglomerado(ultimos_individuos_aglomerados, col_pondera=9, col_nivel_edu=26):

    grupos = ultimos_individuos_aglomerados

    for aglo, filas_aglo in grupos.items():

        total_validos = suma_ponderada(filas_aglo, lambda f: f[col_nivel_edu] not in ['7', '9'], col_pondera)
        total_universitarios = suma_ponderada(filas_aglo, lambda f: f[col_nivel_edu] in ['5', '6'], col_pondera)
        
        nombre = nombre_aglomerado(aglo)
        
        if total_validos > 0:

            print(f"{nombre}: {porcentaje(total_universitarios,total_validos):.2f}% con nivel universitario o superior")
        
        else:
            print(f"{nombre}: sin datos válidos")

In [109]:
porcentaje_universitarios_por_aglomerado(ultimos_individuos_aglomerados)

Ciudad Autónoma de Buenos Aires: 49.65% con nivel universitario o superior
Partidos del GBA: 23.12% con nivel universitario o superior
Gran Catamarca: 26.14% con nivel universitario o superior
Jujuy - Palpalá: 29.16% con nivel universitario o superior
Gran Córdoba: 34.33% con nivel universitario o superior
Río Cuarto: 31.10% con nivel universitario o superior
Concordia: 19.40% con nivel universitario o superior
Formosa: 24.87% con nivel universitario o superior
Gran Resistencia: 24.16% con nivel universitario o superior
Comodoro Rivadavia - Rada Tilly: 25.36% con nivel universitario o superior
La Rioja: 25.55% con nivel universitario o superior
Rawson - Trelew: 22.06% con nivel universitario o superior
Viedma - Carmen de Patagones: 27.73% con nivel universitario o superior
Mar del Plata: 31.72% con nivel universitario o superior
San Nicolás - Villa Constitución: 23.74% con nivel universitario o superior
Gran Paraná: 33.14% con nivel universitario o superior
Santa Rosa - Toay: 27.27% co

### Consulta 8
8. Informar las regiones en orden descendente según el porcentaje de inquilinos de
cada una

In [110]:
def informar_porcentaje_inquilinos_por_aglomerado(ultimos_hogares_aglomerados):
    """
    Informa los aglomerados ordenados de forma descendente según el porcentaje de inquilinos
    para un período determinado (año y trimestre).
        
    Retorna:
    - Lista de tuplas: [(nombre_aglomerado, porcentaje_inquilinos), ...]
    """

    resultados = []
    aglomerados = ultimos_hogares_aglomerados

    for codigo, hogares in aglomerados.items():
        total_ponderado = suma_ponderada(hogares, lambda f: True, 8) # lambda f: True para que sume todos sin filtrar nada
        total_inquilinos = suma_ponderada(hogares, lambda f: f[37] == '3', 8) # lambda f: f[37] == '3' solo pondera cuando f[37] == '3' 

        if total_ponderado > 0:
            porcentaje_inq = (total_inquilinos/ total_ponderado) * 100
            nombre = nombre_aglomerado(codigo)
            resultados.append((nombre, porcentaje_inq))
        codigo_1 = codigo 
        hogares_1 = hogares
   

    # Ordena en forma descendente por porcentaje
    resultados.sort(key=lambda x: x[1], reverse=True)

    return resultados

In [111]:
res = informar_porcentaje_inquilinos_por_aglomerado(ultimos_hogares_aglomerados)

for nombre, porc in res:
    print(f"{nombre}: {porc:.2f}% de inquilinos")

Ushuaia - Río Grande: 36.07% de inquilinos
Río Gallegos: 35.81% de inquilinos
Ciudad Autónoma de Buenos Aires: 35.15% de inquilinos
Río Cuarto: 34.49% de inquilinos
Gran Córdoba: 30.64% de inquilinos
Bahía Blanca - Cerri: 28.26% de inquilinos
Gran Mendoza: 26.90% de inquilinos
Santa Rosa - Toay: 24.23% de inquilinos
Comodoro Rivadavia - Rada Tilly: 23.61% de inquilinos
Gran San Luis: 23.27% de inquilinos
Gran Santa Fé: 23.21% de inquilinos
Rawson - Trelew: 22.94% de inquilinos
Neuquén - Plottier: 22.90% de inquilinos
Mar del Plata: 22.66% de inquilinos
Viedma - Carmen de Patagones: 22.46% de inquilinos
Corrientes: 21.49% de inquilinos
Gran Paraná: 21.05% de inquilinos
Gran Rosario: 19.94% de inquilinos
Gran Salta: 19.85% de inquilinos
Posadas: 19.17% de inquilinos
Gran La Plata: 19.07% de inquilinos
Gran Tucumán - Tafí Viejo: 18.10% de inquilinos
Gran San Juan: 17.07% de inquilinos
La Rioja: 15.29% de inquilinos
Partidos del GBA: 15.15% de inquilinos
San Nicolás - Villa Constitución: 1

### Consulta 9
9. Pedir al usuario que seleccione un aglomerado y a partir de la información contenida
retornar una tabla que contenga la cantidad de personas mayores de edad según su
nivel de estudios alcanzados

A modo de ejemplo:
| Año  | Trimestre | Primario incompleto | Primario completo | Secundario incompleto | Secundario completo | Superior o universitario |
|------|-----------|----------------------|--------------------|------------------------|----------------------|----------------------------|
| 2020 | 1         |                      |                    |                        |                      |                            |
| 2020 | 2         |                      |                    |                        |                      |                            |
| 2020 | 3         |                      |                    |                        |                      |                            |
| 2020 | 4         |                      |                    |                        |                      |                            |
| 2021 | 1         |                      |                    |                        |                      |                            |
| 2021 | 2         |                      |                    |                        |                      |                            |


In [112]:
def tabla_porcentaje_secundario_incompleto(filas_aglomerado, aglomerado):
    
    tabla = {}

    # Filtrar por aglomerado
    filas_por_aglo = filas_aglomerado[aglomerado]
    # Filtrar por mayores de edad (edad >= 18)
    filas_filtradas = list(filter(lambda f: int(f[13]) >= 18, filas_por_aglo, ))

    #  Contar según nivel educativo para cada año y trimestre
    for fila in filas_filtradas:
        anio = int(fila[1])        # Año
        trimestre = int(fila[2])   # Trimestre
        pondera = float(fila[9])   # Factor de expansión
        nivel = fila[26]           # Nivel educativo

        clave = (anio, trimestre)
        if clave not in tabla:
            tabla[clave] = {
                'Primario incompleto': 0,
                'Primario completo': 0,
                'Secundario incompleto': 0,
                'Secundario completo': 0,
                'Superior o universitario': 0
            }

        
        if nivel == '1':
            tabla[clave]['Primario incompleto'] += pondera

        elif nivel == '2':
            tabla[clave]['Primario completo'] += pondera

        elif nivel == '3':
            for elem in ['Primario completo',"Secundario incompleto"]:
                tabla[clave][elem] += pondera

        elif nivel == '4':
            for elem in ['Primario completo',"Secundario completo"]:
                tabla[clave][elem] += pondera
                
        elif nivel in ['5', '6']:
            for elem in ['Primario completo',"Secundario completo","Superior o universitario"]:
                tabla[clave][elem] += pondera
                

    # Imprimir la tabla
    print(f"\nTabla de personas mayores de edad por nivel educativo en el aglomerado {aglomerado}:\n")
    print("| {:<6} | {:<9} | {:>22} | {:>20} | {:>24} | {:>22} | {:>28} |".format(
        "Año", "Trimestre", "Primario incompleto", "Primario completo",
        "Secundario incompleto", "Secundario completo", "Superior o universitario"))
    print("|" + "-"*8 + "|" + "-"*11 + "|" + "-"*24 + "|" + "-"*22 + "|" + "-"*26 + "|" + "-"*24 + "|" + "-"*30 + "|")

    for clave in sorted(tabla.keys()):
        año, trim = clave
        fila = tabla[clave]
        print("| {:<6} | {:<9} | {:>22.0f} | {:>20.0f} | {:>24.0f} | {:>22.0f} | {:>28.0f} |".format(
            año, trim,
            fila['Primario incompleto'],
            fila['Primario completo'],
            fila['Secundario incompleto'],
            fila['Secundario completo'],
            fila['Superior o universitario']))
        


In [113]:
aglomerado = input("ingrese el aglomerado a buscar") 
tabla_porcentaje_secundario_incompleto(aglomerados_individuos,aglomerado)

KeyError: 'l'

### Consulta 10
10. Pedir al usuario que seleccione dos aglomerados y a partir de la información
contenida retornar una tabla que contenga el porcentaje de personas mayores de
edad con secundario incompleto.


In [ ]:
# 1. Contador por año y trimestre
def contar_secundario_incompleto(filas):
    resultados = {}
    for f in filas:
        año = int(f[1])
        trimestre = int(f[2])
        pondera = float(f[9])
        nivel = f[40]  # Nivel educativo

        clave = (año, trimestre)
        if clave not in resultados:
            resultados[clave] = {'total': 0, 'incompleto': 0}

        resultados[clave]['total'] += pondera
        if nivel in ['1', '2', '3']:  # No terminó el secundario
            resultados[clave]['incompleto'] += pondera

    return resultados


# 2. Calcula porcentajes por aglomerado
def calcular_porcentajes_por_aglomerado(filas_individual, aglo1, aglo2):
    # Filtramos mayores de edad y por aglomerado
    filas_mayores = [f for f in filas_individual if int(f[13]) >= 18]

    filas_aglo1 = [f for f in filas_mayores if int(f[8]) == aglo1]
    filas_aglo2 = [f for f in filas_mayores if int(f[8]) == aglo2]

    # Usamos la función auxiliar para contar
    datos_aglo1 = contar_secundario_incompleto(filas_aglo1)
    datos_aglo2 = contar_secundario_incompleto(filas_aglo2)

    # Unimos claves para poder comparar año/trimestre
    claves = set(datos_aglo1.keys()).union(datos_aglo2.keys())
    resultados = {}

    for clave in sorted(claves):
        total1 = datos_aglo1.get(clave, {'total': 0, 'incompleto': 0})
        total2 = datos_aglo2.get(clave, {'total': 0, 'incompleto': 0})

        porc1 = (total1['incompleto'] / total1['total'] * 100) if total1['total'] > 0 else 0
        porc2 = (total2['incompleto'] / total2['total'] * 100) if total2['total'] > 0 else 0

        resultados[clave] = (porc1, porc2)

    return resultados


# 3. Mostrar tabla final
def mostrar_tabla_porcentajes(resultados, aglo1, aglo2):
    nombre1 = nombre_aglomerado(str(aglo1))
    nombre2 = nombre_aglomerado(str(aglo2))

    print(f"\nComparación de % de personas mayores sin secundario completo")
    print(f"Aglomerado {aglo1}: {nombre1}  vs  Aglomerado {aglo2}: {nombre2}\n")
    print("| {:<6} | {:<9} | {:>30} | {:>30} |".format(
        "Año", "Trimestre", f"% sin secundario ({nombre1})", f"% sin secundario ({nombre2})"))
    print("|" + "-"*8 + "|" + "-"*11 + "|" + "-"*32 + "|" + "-"*32 + "|")

    for (año, trim), (p1, p2) in sorted(resultados.items()):
        print("| {:<6} | {:<9} | {:>30.2f} | {:>30.2f} |".format(año, trim, p1, p2))



In [ ]:
resultados = calcular_porcentajes_por_aglomerado(lista_filas_individual, 12, 13)
mostrar_tabla_porcentajes(resultados, 12, 13)


Comparación de % de personas mayores sin secundario completo
Aglomerado 12: Corrientes  vs  Aglomerado 13: Gran Córdoba

| Año    | Trimestre |  % sin secundario (Corrientes) | % sin secundario (Gran Córdoba) |
|--------|-----------|--------------------------------|--------------------------------|
| 2023   | 3         |                          40.75 |                          32.63 |
| 2023   | 4         |                          40.83 |                          31.83 |
| 2024   | 1         |                          41.22 |                          32.54 |
| 2024   | 2         |                          40.56 |                          32.49 |


### Consulta 11
11. Pedir al usuario que seleccione un año, y busque en el último trimestre almacenado
del mencionado año, el aglomerado con mayor porcentaje de viviendas de “Material
precario” y el aglomerado con menor porcentaje de viviendas de “Material precario”.

In [ ]:
def buscar_max_trimestre (anios_trimestres_hogares,anio):
    return max(anios_trimestres_hogares[anio].keys())

def analizar_material_aglomerados (anios_trimestres_hogares, anio_ingresado):
    
    trimestre = buscar_max_trimestre(anios_trimestres_hogares,anio_ingresado)

    filtrar_mat_precario = list(filter( lambda f : f[89] == "Material precario" , anios_trimestres_hogares[anio_ingresado][trimestre]))

    aglomerados_m_precario = {}
    
    aglomerados_total = {}

    aglomerados_porcentaje = {}
 
    for elem in anios_trimestres_hogares[anio_ingresado][trimestre]:

        pondera = int(elem[7])

        if elem in filtrar_mat_precario:

            if elem[7] not in aglomerados_m_precario:
                aglomerados_m_precario[elem[7]] = 0

            aglomerados_m_precario[elem[7]] += pondera
       
        if elem[7] not in aglomerados_total:
            aglomerados_total[elem[7]] = 0
        aglomerados_total[elem[7]] += pondera

        if elem[7] not in aglomerados_porcentaje:
            aglomerados_porcentaje[elem[7]] = 0.0
 
    for aglomerado in aglomerados_m_precario:
        m_precarios = int(aglomerados_m_precario[aglomerado])
        total = int(aglomerados_total[aglomerado])
        aglomerados_porcentaje[aglomerado] = porcentaje(m_precarios, total )
    
    valor_min = 100.0 
    valor_max = 0.0
    for aglomerado in aglomerados_m_precario:
        if aglomerados_porcentaje[aglomerado] >= valor_max: 
            valor_max = aglomerados_porcentaje[aglomerado]
            clave_max = aglomerado
        
        if aglomerados_porcentaje[aglomerado] <= valor_min: 
            valor_min = aglomerados_porcentaje[aglomerado]
            clave_min = aglomerado


    print(f'en el año {anio_ingresado} , trimestre {trimestre}')
    print(f'el aglomerado con mas cantidad de viviendas con material precario fue:  {clave_max} con {valor_max:.2f}%')
    print(f'el aglomerado con menor cantidad de viviendas con material precario fue: {clave_min} con {valor_min:.2f}%')



In [ ]:
anio_ingresado = input("ingrese el año a analizar")
analizar_material_aglomerados(anios_trimestres_hogares,anio_ingresado)

en el año 2024 , trimestre 2
el aglomerado con mas cantidad de viviendas con material precario fue:  27 con 10.83%
el aglomerado con menor cantidad de viviendas con material precario fue: 32 con 0.16%


### Consulta 12  
12. A partir de la información del último trimestre almacenado en el sistema se debe
retornar para cada aglomerado el porcentaje de jubilados que vivan en una vivienda
con CONDICION_DE_HABITABILIDAD insuficiente

In [121]:
def activ_12 (individuos, hogares): 

    jubilados = {}
    insuficientes = {}
    porcentajes_aglo = {}
    insuficientes_CODUSU = []

    for aglomerado in individuos:
        
        if aglomerado not in porcentajes_aglo: 
            jubilados[aglomerado] = []

        for elem in individuos[aglomerado]:
            if elem[29] == "1":
                jubilados[aglomerado].append(elem)

        for elem in hogares[aglomerado]: 
            if elem[len(elem)-1] == "insuficiente": 
                insuficientes_CODUSU.append(elem[0])

    for aglomerado in jubilados:
        
        total = jub_insuficientes = 0 

        for elem in jubilados[aglomerado]: 
            pondera = int(elem[8])
            
            if elem[0] in insuficientes_CODUSU: 
                jub_insuficientes += pondera 

            total += pondera

        if aglomerado not in porcentajes_aglo:
            porcentajes_aglo[aglomerado] = 0.0
        porcentajes_aglo[aglomerado] = porcentaje(jub_insuficientes,total)

    print(f'la cantidad de jubilados en viviendas insuficientes para cada aglomerado son: ')
    for aglomerado in porcentajes_aglo: 
        print(f'{aglomerado} : {porcentajes_aglo[aglomerado]}')
        

In [122]:
activ_12(ultimos_individuos_aglomerados,ultimos_hogares_aglomerados)    

la cantidad de jubilados en viviendas insuficientes para cada aglomerado son: 
32 : 0.0
33 : 0.0
22 : 0.0
19 : 0.0
13 : 0.0
36 : 0.0
14 : 0.0
15 : 0.0
8 : 0.9803921568627451
9 : 0.0
25 : 0.0
91 : 0.0
93 : 0.0
34 : 0.0
38 : 0.0
6 : 0.0
30 : 0.0
3 : 0.0
12 : 0.0
10 : 0.0
7 : 0.0
17 : 0.0
23 : 0.0
5 : 0.0
27 : 0.0
29 : 0.0
4 : 0.0
31 : 0.0
26 : 0.0
20 : 0.0
18 : 0.0
2 : 0.0


In [ ]:
archivo_hogares.close()
archivo_individuos.close()